# 아주 간단한 추천시스템 만들기

아주 간단한 추천 시스템을 만들고, **추천 시스템을 평가하는 방법**에 대해서 배웁니다. 여기에서 pandas 데이터 프레임 사용을 아주 익숙하게 만들어야 할 것 같고요. 간단한 평가 메트릭의 원리와 구현을 설명합니다. 
데잇걸즈 강의에서는 RMSE를 직접 구현하는걸 만들었는데, 여기에서는 **개념 설명** 해주고 + **파이썬으로 구현**하는건 보여만주고 + **계산은 sklearn.metrics에 mean_squared_error** 사용하는게 좋을 것 같아요.

1) 가장 간단한 예측하기

- 전체 데이터의 평균 평점으로 예측하기

- 추천 시스템 평가하기

2) 유저의 특성을 반영하여 예측하기

- 각 유저의 평균 평점으로 예측하기

- 추천 시스템 평가하기

3) 아이템의 특성을 반영하여 예측하기

- 각 아이템의 평균 평점으로 예측하기

- 추천 시스템 평가하기

In [1]:
%matplotlib inline

import pandas as pd

## 데이터 셋팅

### 데이터 불러오기

In [2]:
clean_MR = pd.read_csv('./data/ml-latest-small/clean_MR.csv')

In [3]:
clean_MR

,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,False,False,False,False,True,False,...,False,False,True,False,False,False,False,1,4.0,964982703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,False,False,False,False,True,False,...,False,False,True,False,False,False,False,5,4.0,847434962
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,False,False,False,False,True,False,...,False,False,True,False,False,False,False,7,4.5,1106635946
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,False,False,False,False,True,False,...,False,False,True,False,False,False,False,15,2.5,1510577970
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995.0,False,False,False,False,True,False,...,False,False,True,False,False,False,False,17,4.5,1305696483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100784,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,2017.0,False,False,False,False,False,False,...,False,False,True,False,False,True,False,184,4.0,1537109082
100785,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,2017.0,False,False,False,False,False,False,...,False,False,True,False,False,False,False,184,3.5,1537109545
100786,193585,Flint (2017),Drama,2017.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,184,3.5,1537109805
100787,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,2018.0,False,False,False,False,False,False,...,False,False,True,False,False,True,False,184,3.5,1537110021


## 예측과 평가

추천시스템을 만들기에 앞서 앞으로 계속해서 반복하게 될 가장 기본적인 프로세스에 대해 설명하겠습니다.

우리가 앞으로 실습을 해볼 것은 영화평점을 예측하는 프로젝트라고 했습니다.
즉, 영화 평점을 예측하는 무수히 많은 방법들 중에서 가장 좋은 방법을 찾는 여정이 될 텐데요.

그런데 어떤 방법이 가장 좋은 방법인지는 무슨 기준으로 판별할 수가 있을까요?

평가 방식이 다르다면 같은 데이터에 대한 두 가지 다른 방식의 예측에 대해 어떨 때는 A라는 예측방식이 더 낫다고 할 수도 있고, B라는 예측방식이 더 낫다고 결론지을 수도 있습니다. 평가 방식은 예측의 방향성을 정해주는 기준이 되기 때문에 이런 종류의 예측 모델을 만들 때에는 예측값을 평가하는 방식을 아는 것도 굉장히 중요합니다.

이번 강의에서는 추천시스템 전반에 대한 이해가 우선이기 때문에 가장 보편적으로 쓰이는 성능평가방식인 RMSE를 사용해보도록 하겠습니다.

### 평가지표 RMSE

RMSE는 Root Mean Square Error의 약자로 정답과 예측값의 차이를 제곱한 값들의 평균을 의미합니다.

정답에서 예측값을 빼면 예측값이 정답보다 큰 경우에는 마이너스의 값이 나오게됩니다. 우리는 예측값들과 정답값 사이의 평균적인 차이에 대해 알고 싶은 것인데 마이너스 값이 나와버리면 다른 오차들과 합쳐서 계산을 하려고 했을 때 값이 상쇄되어버리겠죠?

그래서 각각의 오차에 제곱을 먼저 해주어서 부호의 영향을 없애 준 후 평균을 내고 다시 제곱근을 씌워 제곱하여 뻥튀기 되었던 값을 원래의 오차값에 비슷하게끔 되돌려 주는 방식이 바로 RMSE 방식입니다.

제곱을 하고 다시 제곱근을 하고 하는 방식이 번거로워 보이실 수도 있겠지만 사실 컴퓨터가 연산하기에는 아주 쉬운 작업입니다. 오히려 컴퓨터는 경우의 수가 나뉘는 것에 대해 처리를 알아서 할 수가 없으므로 차라리 여러번 과정을 거치더라도 모든 경우에 통하는 방법이 계산이 더 쉬운 것이지요.

### RMSE의 특징
RMSE는 비교적 설명이 쉽습니다.

RMSE는 오차를 제곱했던 것에 다시 제곱근을 씌워 원래의 데이터가 가지고 있던 스케일을 복원시켜주므로 RMSE 값의 단위는 원래 데이터가 가지고 있는 단위와 일치합니다. 

즉, 학생의 평균 신장을 예측하는 프로젝트를 한다고 했을 때, 학생들의 키를 측정하는 기준이 cm였고. RMSE를 이용해 모델의 성능을 평가한 결과 RMSE값이 15가 나왔다면.

우리가 구한 예측값이 평균적으로 정답에서 15(cm) 값 만큼 떨어져있다(오차가 난다) 정도로 해석 해볼 수 있겠습니다.

### 파이썬으로 RMSE 직접 구현해보기

다시 영화 평점으로 돌아와서, 실제 숫자들을 이용해 RMSE를 한 번 직접 구해보며 이해 해보도록 하겠습니다.

In [4]:
# 수식 계산을 위해 numpy 패키지 임포트

import numpy as np

In [5]:
# 실제 유저의 평점 : 4.5
# 예측 평점 : 3

print(4.5 - 3)

print((4.5 - 3)**2)

print(np.mean((4.5 - 3)**2))

print(np.sqrt(np.mean((4.5 - 3)**2)))

1.5
2.25
2.25
1.5


In [6]:
# 실제 유저의 평점 : 4.5
# 예측 평점 : 4

np.sqrt(np.mean((4.5 - 4) **2 ))

0.5

In [7]:
# 실제 유저의 평점 : 4.5
# 예측 평점 : 4.5

np.sqrt(np.mean((4.5 - 4.5) **2 ))

0.0

In [8]:
# 실제 유저의 평점 : 4.5
# 예측 평점 : 5

np.sqrt(np.mean((4.5 - 5) **2 ))

0.5

위 코드들의 결과를 보면 실제값 = 예측값일 때 RMSE 값이 가장 작은 것을 알 수가 있습니다.

그럼 이번에는 우리가 예측한 모든 값을 이용해서 RMSE를 구하는 함수를 만들어보겠습니다.

In [9]:
def RMSE(answers, predictions):
    SE_list = []
    
    for i in range(0, len(answers)):
        SE_list.append((answers[i] - predictions[i])**2)
    
    return np.sqrt(np.mean(SE_list))

위에서 구해보았던 값들을 가지고 함수가 잘 작동하는지 테스트를 해보겠습니다.

In [10]:
RMSE([4.5], [3])

1.5

In [11]:
RMSE([4.5], [4.5])

0.0

이번엔 네개의 정답과 네개의 예측값을 넣어볼까요.

In [12]:
RMSE([1,2,3,4], [0,0,0,0])

2.7386127875258306

In [13]:
RMSE([1,2,3,4], [1,2,3,4])

0.0

### scikit-learn 패키지의 RMSE 함수 사용해보기

자 열심히 RMSE를 만들어봤는데요. RMSE는 사실 이렇게 한땀한땀 만들 필요는 없습니다.

scikit-learn이라고 하는 패키지 내에 MSE를 구하는 기능이 이미 탑재가 되어있는데요. 이 패키지에 옵션을 잘 넣어주면 RMSE도 손쉽게 구해줍니다.

1. **sklearn 버전이 0.21.x 이하**인 경우에는 MSE를 RMSE로 바꿔주는 옵션이 내장되어있지 않습니다.
따라서 해당 함수를 이용하여 MSE를 먼저 구하고 루트를 계산해주는 방식으로 RMSE를 구해야합니다.

- `np.sqrt(mean_squared_error(y_true, y_pred))`


2. **sklearn 버전이 0.22.x일 경우**에는 MSE를 RMSE로 바꾸어주는 squared 옵션이 있습니다.

- `mean_squared_error(y_true, y_pred, squared=False)`

이 옵션을 True로 하게되면 그냥 MSE를 구하게 되고 이 옵션을 False로 두게되면 RMSE를 구해줍니다.

두번째 방법을 시도해보시다가 `mean_squared_error() got an unexpected keyword argument 'squared'`라는 에러메세지를 마주하셨다면 1번 방법으로 계산해보세요.

In [14]:
import sklearn
sklearn.__version__

'0.21.3'

In [15]:
# sklearn 패키지에서 mean_squared_error 임포트
from sklearn.metrics import mean_squared_error

# 1. mean_squared_error(y_true, y_pred)**1/2
# 2. mean_squared_error(y_true, y_pred, squared=False)
np.sqrt(mean_squared_error([4.5], [3]))

1.5

RMSE가 무엇인지 이해하셨다면 귀찮게 RMSE 함수 만드실 필요 없이 간단하게 scikit-learn 패키지를 이용하셔서 RMSE를 구해보시면 됩니다.

팀에서 한 두명이 완벽하게 정답을 맞췄다고 해도 다른 팀원들이 정말 말도 안되는 수로 예측을 했다면 전체적인 팀 성적이 안좋아지는 것은 당연하겠죠.

RMSE는 가장 직관적이고 구하기도 비교적 쉬운 편이라서 데이터 전반적으로 머신러닝 모델의 예측 성능이 어느정도 되는지 확인해보기에 좋은 성능 평가 지표라고 할 수 있겠습니다.

RMSE 이외에도 문제에 따라 목표하는 것에 따라 성능지표는 얼마든지 바꿀 수 있고 또 그에 맞춰서 모델을 개발해나갈 수 있습니다. 기회가 된다면 성능 지표 부분에 대해서도 다루어볼 예정이지만 이번 강의에서는 성능지표 보다는 추천 시스템 개발에 초첨을 맞추었기 때문에 우선 이정도만 알고 넘어가보겠습니다.

## 가장 간단한 예측하기

자 이제 데이터도 준비가 되었고, 우리가 영화 평점을 예측했을 때 그게 얼마나 잘 예측한 것인지 평가해 줄 방법도 찾았습니다. 이제 예측만 하면 되겠는데요.

사실 평점을 예측한다는 것을 엄밀하게 따져보자면 그렇게 어려운 일이 아닙니다. 이렇게 거창한 강의 듣지 않아도 누구든지 할 수 있죠. 여러분이 너무 어렵게만 생각해서 그런데, 아주 간단하게라도 예측해보는 방법은 어디에나 존재합니다. 다만 그렇게 하는게 꺼려질 뿐이죠. 그렇게하면 아마도 좋은 결과가 안 나올거라고 생각해서 그렇습니다. 진짜 그럴까요?

오늘은 추천시스템이고 뭐고 아무것도 모를 때 가장 아무생각없이 예측해볼 수 있는 방법들을 한 번 시도해보도록 하겠습니다.

아, 그전에 먼저 해두어야 할 작업이 있습니다.

## Training set / Test set 분리하기

데이터를 기반해서 뭔가를 예측해보기 위해서는 우선 예측의 근거가 될 데이터들과 예측을 해야하는 데이터들이 있어야 합니다.

예측의 근거가 될 데이터들은 예측 모델을 트레이닝 시키는 데이터라고 해서 트레이닝 세트라고 하며, 예측을 해야하는 데이터들은 예측결과를 테스트하는 데이터들이라고 해서 테스트 세트라고 부릅니다.

이 트레이닝, 테스트 데이터를 어떻게 나누고, 어떤 비율로 나눌 것인가하는 것은 임의로 정하면 되는 부분인데요.

우리는 이번에 timestamp를 기준으로 가장 최근에 입력된 100개의 데이터를 테스트 셋으로 설정하고 나머지 데이터를 모두 트레이닝 셋으로 설정해보도록 하겠습니다.

In [16]:
# timestamp 기준 오름차순 정렬

clean_MR.sort_values('timestamp', inplace=True)

In [17]:
# 최신 데이터 100개를 테스트 셋트로

test_set = clean_MR[-100:]
training_set = clean_MR[:-100]

In [18]:
# test_set의 ratings 값만 answers에 따로 저장

answers = test_set['rating'].values
answers

array([5. , 4.5, 5. , 3. , 3. , 3. , 3. , 2.5, 4. , 3.5, 4.5, 5. , 3. ,
       1.5, 3. , 4. , 3.5, 5. , 4. , 3. , 3. , 2. , 3.5, 4. , 4. , 4. ,
       2. , 4. , 4. , 3. , 3.5, 5. , 4. , 2.5, 3. , 4. , 5. , 3. , 3. ,
       2.5, 5. , 2.5, 3. , 2. , 4. , 2. , 5. , 4. , 4. , 2.5, 3.5, 3. ,
       4. , 2. , 5. , 0.5, 5. , 2. , 2. , 3. , 3. , 1. , 4. , 5. , 3. ,
       3. , 3. , 2.5, 3. , 3.5, 2. , 3. , 4.5, 5. , 5. , 5. , 5. , 3.5,
       4. , 4. , 4. , 4. , 4.5, 5. , 3.5, 2.5, 4.5, 4. , 5. , 4.5, 4.5,
       4. , 4. , 4. , 4. , 2.5, 3. , 2.5, 1.5, 4. ])

In [19]:
# test_set의 ratings 0으로 초기화

test_set['rating'] = 0
test_set

/home/leebom/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
96703,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,2014.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,0,1537158021
76920,8784,Garden State (2004),Comedy|Drama|Romance,2004.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,0,1537158023
97672,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2015.0,False,False,True,True,False,False,...,False,False,False,True,False,False,False,331,0,1537158030
87268,59369,Taken (2008),Action|Crime|Drama|Thriller,2008.0,False,False,True,False,False,False,...,True,False,False,True,False,True,False,331,0,1537158039
80309,34162,Wedding Crashers (2005),Comedy|Romance,2005.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,331,0,1537158043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100732,187031,Jurassic World: Fallen Kingdom (2018),Action|Adventure|Drama|Sci-Fi|Thriller,2018.0,False,False,True,True,True,False,...,False,False,False,True,False,True,False,514,0,1537674927
100753,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,False,False,False,True,True,False,...,False,False,False,False,False,True,False,514,0,1537674946
66124,5247,Smokey and the Bandit (1977),Action|Comedy,1977.0,False,False,False,False,False,False,...,False,False,False,False,False,True,False,514,0,1537757040
66116,5246,Smokey and the Bandit II (1980),Action|Comedy,1980.0,False,False,False,False,False,False,...,False,False,False,False,False,True,False,514,0,1537757059


## 100명의 유저의 평점을 한개의 숫자로 예측해보기

가장 간단하게 예측을 해보겠습니다.

### 모두 0점을 주었다고 예측해보기

In [20]:
# test_set 그대로 입니다. 따로 해줄 것이 없네요.

test_set.head()

,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
96703,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,2014.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,0,1537158021
76920,8784,Garden State (2004),Comedy|Drama|Romance,2004.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,0,1537158023
97672,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2015.0,False,False,True,True,False,False,...,False,False,False,True,False,False,False,331,0,1537158030
87268,59369,Taken (2008),Action|Crime|Drama|Thriller,2008.0,False,False,True,False,False,False,...,True,False,False,True,False,True,False,331,0,1537158039
80309,34162,Wedding Crashers (2005),Comedy|Romance,2005.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,331,0,1537158043


100명의 데이터를 모두 0점으로 예측했을 때의 RMSE 점수

In [21]:
# RMSE 확인

np.sqrt(mean_squared_error(answers, test_set['rating']))

3.6834087473426025

3.683...이 나왔군요.

RMSE의 특징에서 RMSE로 구한 값은 원래 데이터의 단위와 같은 단위를 쓸 수 있다고 했습니다.
그렇다면 100개의 데이터에서 모든 영화에 rating이 0일 것이라고 예측한 우리의 첫번째 모델은 평균적으로 정답과 3.683점 오차가 난다고 해석해볼 수 있겠습니다.

실제로는 5점을 준 영화를 0점으로 예측했을 때의 최대로 나오는 오차가 5이라는 것을 생각해보았을 때 3.683점은 썩 좋은 점수는 아닌 것 같습니다. 그럼 다른 방식으로 조금 더 예측을 해볼까요?

### 모두 5점을 주었다고 예측해보기

In [22]:
# test_set의 rating을 전부 5로 변환합니다.

test_set['rating'] = 5
test_set.head()

/home/leebom/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
96703,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,2014.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,5,1537158021
76920,8784,Garden State (2004),Comedy|Drama|Romance,2004.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,5,1537158023
97672,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2015.0,False,False,True,True,False,False,...,False,False,False,True,False,False,False,331,5,1537158030
87268,59369,Taken (2008),Action|Crime|Drama|Thriller,2008.0,False,False,True,False,False,False,...,True,False,False,True,False,True,False,331,5,1537158039
80309,34162,Wedding Crashers (2005),Comedy|Romance,2005.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,331,5,1537158043


In [23]:
# RMSE 확인

np.sqrt(mean_squared_error(answers, test_set['rating']))

1.7937391114652097

모든 영화를 5점으로 예측했더니 RMSE 값이 3.683...에서 1.793...까지 내려갔습니다. 

전부 다 5점으로 예측하기만 했는데 평균적인 오차가 많이 줄어든 것을 보니 100개의 데이터에는 0점에 가까운 값보다는 5점에 가까운 값들이 많이 들어가 있는 것이 아닐까 추측을 해볼 수 있을 것 같습니다.

### 전체 데이터의 평균 평점으로 예측하기

한가지 숫자로 때려 맞춰보는 것은 이제 그만 하고 한번 Training Set의 데이터들을 이용해보겠습니다.

이번에도 가장 쉽게 시작해보죠. Training Set의 모든 데이터의 전체 평균을 구해서 그 값으로 한 번 모든 평점을 채워보도록 하겠습니다.

In [24]:
# training set의 평점 평균 구하기

training_mean = training_set['rating'].mean()
training_mean

3.5015294620067734

In [25]:
# test_set의 rating을 training set의 평점 평균값으로 채워 넣기

test_set['rating'] = training_mean
test_set.head()

/home/leebom/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
96703,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,2014.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,3.501529,1537158021
76920,8784,Garden State (2004),Comedy|Drama|Romance,2004.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,3.501529,1537158023
97672,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2015.0,False,False,True,True,False,False,...,False,False,False,True,False,False,False,331,3.501529,1537158030
87268,59369,Taken (2008),Action|Crime|Drama|Thriller,2008.0,False,False,True,False,False,False,...,True,False,False,True,False,True,False,331,3.501529,1537158039
80309,34162,Wedding Crashers (2005),Comedy|Romance,2005.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,331,3.501529,1537158043


In [26]:
# RMSE 확인

np.sqrt(mean_squared_error(answers, test_set['rating']))

1.0355651968435189

이제 RMSE가 1.035..점이 되었습니다. 맨 처음에 예측했던 값의 RMSE 3.683..점에 비하면 오차가 많이 적어졌네요.

예측값의 평균적인 오차가 1점 정도라는 뜻입니다. 이것보다 더 적은 오차를 가지게 하려면 어떻게 하는 게 좋을까요.

## 데이터별 특성을 반영하여 예측하기

지금까지는 100개의 데이터를 전부 한 개의 숫자로 예측하는 방법을 시도해보았습니다. 사실 이런건 조금만 생각해봐도 말이 안되는 방법이죠?

사람마다 별점을 주는 방식이 다르고 영화들도 각각 다른데 말입니다. 이번에는 100개의 데이터를 조금 더 자세히 들여다보고 예측을 해보도록 하겠습니다.

### 각 유저의 평균으로 예측하기

In [27]:
# training set을 이용하여 각 user의 평균 평점을 구해봅니다.

user_mean = training_set.groupby('userId')['rating'].mean()
user_mean

userId
1      4.366379
2      3.948276
3      2.435897
4      3.555556
5      3.636364
         ...   
606    3.657399
607    3.786096
608    3.134176
609    3.270270
610    3.688556
Name: rating, Length: 610, dtype: float64

In [28]:
# 유저들의 평균 평점을 구해놓은 데이터에서 test_set의 user_id값 별로 평균 평점을 가져옴
# 만약 training에는 없고 test에만 있는 user일 경우에는 평점을 트레이닝 전체 데이터의 평균으로 채워 넣음
test_set['rating'] = test_set.apply(lambda x: user_mean[x['userId']] if x['userId'] in user_mean.index else training_mean, axis=1)
test_set

/home/leebom/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
96703,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,2014.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,3.764368,1537158021
76920,8784,Garden State (2004),Comedy|Drama|Romance,2004.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,3.764368,1537158023
97672,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2015.0,False,False,True,True,False,False,...,False,False,False,True,False,False,False,331,3.764368,1537158030
87268,59369,Taken (2008),Action|Crime|Drama|Thriller,2008.0,False,False,True,False,False,False,...,True,False,False,True,False,True,False,331,3.764368,1537158039
80309,34162,Wedding Crashers (2005),Comedy|Romance,2005.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,331,3.764368,1537158043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100732,187031,Jurassic World: Fallen Kingdom (2018),Action|Adventure|Drama|Sci-Fi|Thriller,2018.0,False,False,True,True,True,False,...,False,False,False,True,False,True,False,514,3.320513,1537674927
100753,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,False,False,False,True,True,False,...,False,False,False,False,False,True,False,514,3.320513,1537674946
66124,5247,Smokey and the Bandit (1977),Action|Comedy,1977.0,False,False,False,False,False,False,...,False,False,False,False,False,True,False,514,3.320513,1537757040
66116,5246,Smokey and the Bandit II (1980),Action|Comedy,1980.0,False,False,False,False,False,False,...,False,False,False,False,False,True,False,514,3.320513,1537757059


In [29]:
# 예측 평점 중 혹시 NULL값이 있는지 확인
# NULL값이 존재하면 mean_squared_error() 함수가 제대로 동작하지 않음

test_set['rating'].isnull().sum()

0

In [30]:
# RMSE 확인

np.sqrt(mean_squared_error(answers, test_set['rating']))

1.0309078039481152

### 각 아이템의 평균 평점으로 예측하기

In [31]:
# training set을 이용하여 각 movie의 평균 평점을 구해봅니다.

movie_mean = training_set.groupby('movieId')['rating'].mean()
movie_mean

movieId
1         3.920930
2         3.431818
3         3.259615
4         2.357143
5         3.071429
            ...   
193581    4.000000
193583    3.500000
193585    3.500000
193587    3.500000
193609    4.000000
Name: rating, Length: 9686, dtype: float64

In [32]:
# 유저들의 평균 평점을 구해놓은 데이터에서 test_set의 user_id값 별로 평균 평점을 가져옴
# 만약 training에는 없고 test에만 있는 user일 경우에는 평점을 트레이닝 전체 데이터의 평균으로 채워 넣음
test_set['rating'] = test_set.apply(lambda x: movie_mean[x['movieId']] if x['movieId'] in movie_mean.index else training_mean, axis=1)
test_set

/home/leebom/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,title,genres,year,Western,Mystery,Drama,Sci-Fi,Adventure,IMAX,...,Crime,Musical,Animation,Thriller,Documentary,Action,War,userId,rating,timestamp
96703,109374,"Grand Budapest Hotel, The (2014)",Comedy|Drama,2014.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,3.754902,1537158021
76920,8784,Garden State (2004),Comedy|Drama|Romance,2004.0,False,False,True,False,False,False,...,False,False,False,False,False,False,False,331,3.691489,1537158023
97672,115713,Ex Machina (2015),Drama|Sci-Fi|Thriller,2015.0,False,False,True,True,False,False,...,False,False,False,True,False,False,False,331,3.870370,1537158030
87268,59369,Taken (2008),Action|Crime|Drama|Thriller,2008.0,False,False,True,False,False,False,...,True,False,False,True,False,True,False,331,3.634146,1537158039
80309,34162,Wedding Crashers (2005),Comedy|Romance,2005.0,False,False,False,False,False,False,...,False,False,False,False,False,False,False,331,3.517544,1537158043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100732,187031,Jurassic World: Fallen Kingdom (2018),Action|Adventure|Drama|Sci-Fi|Thriller,2018.0,False,False,True,True,True,False,...,False,False,False,True,False,True,False,514,4.000000,1537674927
100753,187595,Solo: A Star Wars Story (2018),Action|Adventure|Children|Sci-Fi,2018.0,False,False,False,True,True,False,...,False,False,False,False,False,True,False,514,4.125000,1537674946
66124,5247,Smokey and the Bandit (1977),Action|Comedy,1977.0,False,False,False,False,False,False,...,False,False,False,False,False,True,False,514,2.750000,1537757040
66116,5246,Smokey and the Bandit II (1980),Action|Comedy,1980.0,False,False,False,False,False,False,...,False,False,False,False,False,True,False,514,2.333333,1537757059


In [33]:
# 예측 평점 중 혹시 NULL값이 있는지 확인
# NULL값이 존재하면 mean_squared_error() 함수가 제대로 동작하지 않음

test_set['rating'].isnull().sum()

0

In [34]:
# RMSE 확인

np.sqrt(mean_squared_error(answers, test_set['rating']))

0.9344391586880029

# 피드백 필요한 부분 

- 평가 방법 위에 넣을지 아래에 넣을지(위에?)
- RMSE 설명 충분한지...
- 혹시 너무 어려운 코드는 없는지
- 순서나 배치나 flow는 괜찮은지

## 예측 결과 평가방법(RMSE)

### 쌩 파이썬으로 구해보는 RMSE

### scikit-learn 패키지의 도움으로 RMSE 구하기

-

-